

##Анализ тематики дневника К.А.Березкина







In [ ]:
!pip install openai

In [ ]:
# Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import json
import requests
import pandas as pd

# Настройка API

PROXY_API_KEY = "PROXY_API_KEY" # Вставьте ваш PROXY_API_KEY

# Настройка путей
base_dir = "/content/drive/My Drive/DIARY"
input_folder = os.path.join(base_dir, 'texts')
data_folder = os.path.join(base_dir, 'data')

# Функция для чтения всех текстовых файлов и создания DataFrame
def create_dataframe_from_files(input_folder):
    files = sorted([f for f in os.listdir(input_folder) if f.endswith('.txt')])
    data = []
    for file in files:
        file_path = os.path.join(input_folder, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        data.append({'filename': file, 'text': text})
    return pd.DataFrame(data)

# Чтение текстов из файлов в папку "texts" и создание DataFrame
df = create_dataframe_from_files(input_folder)

# Сохранение DataFrame в CSV файл
df.to_csv(os.path.join(data_folder, 'diary_dataframe.csv'), index=False, encoding='utf-8')
print("DataFrame сохранен в файл 'diary_dataframe.csv'.")

# Системный промпт
system_prompt = """
Вы — эксперт по истории Российской Империи XIX века и специалист по анализу личных дневников и мемуаров со знанием культурных, исторических и социальных аспектов этого периода. Ваша роль заключается в анализе текста дневника и идентификации ключевых тем с подробными объяснениями.

Контекст:
- Автор дневника: Кирилл Антонович Березкин, гимназист Вологодской губернской гимназии, потомственный дворянин.
- Период ведения дневника: январь 1849 - сентябрь 1849.
- Место: город Вологда, Вологодская губерния.
- Историческая значимость: дневник отражает восприятие важных исторических событий в российской провинции.

План действий:
1. Прочитайте предоставленный текст дневника и оцените его содержание.
2. Определите ключевые темы, о которых пишет автор, основываясь на частоте упоминаний, значимости для автора и эмоциональной окраске.
3. Для каждой темы составьте краткое название и подробное описание, учитывающее исторический и культурный контекст середины XIX века в Вологде.
4. Найдите и приведите примеры в виде цитат (с указанием даты записи), которые подтверждают наличие этой темы.
5. Убедитесь, что описание каждой темы детально отражает её значимость и связь с социальной, образовательной или культурной жизнью того времени.

Методические рекомендации:
- Подходите к анализу с междисциплинарной точки зрения, связывая темы с историей, социологией и педагогикой.
- Выбирайте цитаты, которые наиболее репрезентативно и ярко иллюстрируют тему.
"""

# Настройка headers для API
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {PROXY_API_KEY}"
}

# Функция для запроса к Gemini
def get_themes_for_chunk(chunk):
    user_prompt = f"""На основе системных инструкций проведите анализ тематики текста дневника гимназиста из Вологды середины XIX века. Определите и опишите все основные темы, обсуждаемые в дневнике. Представьте результаты в формате списка по следующей схеме:

- **Название темы**: [краткое название темы]
- **Описание**: [подробное описание темы (100 слов), включая её значимость и возможное отражение культурных и социальных реалий того времени]
- **Пример**: [2-3 цитаты из текста дневника со ссылкой на дату записи, которые иллюстрируют эту тему и подчеркивают её значимость]

Повторите эту схему для каждой найденной темы.

{chunk}"""

    # Создаем запрос к Gemini
    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": system_prompt + "\n\n" + user_prompt}
                ]
            }
        ],
        "generationConfig": {
            "temperature": 0.3,
            "maxOutputTokens": 8024
        }
    }

    # Отправка запроса
    response = requests.post(
        "https://api.proxyapi.ru/google/v1/models/gemini-1.5-pro:generateContent",
        headers=headers,
        data=json.dumps(payload)
    )

    # Обработка ответа
    if response.status_code == 200:
        result = response.json()
        return result['candidates'][0]['content']['parts'][0]['text']
    else:
        print(f"Ошибка: {response.status_code}, {response.text}")
        return ""

# Файл для сохранения промежуточных результатов
intermediate_results_path = os.path.join(data_folder, 'intermediate_themes_gemini.txt')

with open(intermediate_results_path, 'w', encoding='utf-8') as f_out:
    themes = []
    for i, row in df.iterrows():
        print(f"Обработка файла {row['filename']} ({i+1}/{len(df)}):")
        chunk = row['text']
        themes_chunk = get_themes_for_chunk(chunk)

        # Сохраняем промежуточный результат
        f_out.write(f"Файл: {row['filename']}\n")
        f_out.write(themes_chunk)
        f_out.write("\n---\n")  # Разделитель

        themes.append(themes_chunk)

# Обобщение всех тем
all_themes = "\n".join(themes)
prompt_summary = f"Вот список ключевых тем нескольких частей одного текста. Обобщите информацию и составте общий список основных тем всего текста с комментариями: \n\n{all_themes}"

# Запрос на обобщение
payload_summary = {
    "contents": [
        {
            "role": "user",
            "parts": [
                {"text": "Вы являетесь экспертом по анализу и обобщению информации." + "\n\n" + prompt_summary}
            ]
        }
    ],
    "generationConfig": {
        "temperature": 0.5,
        "maxOutputTokens": 8024
    }
}

response_summary = requests.post(
    "https://api.proxyapi.ru/google/v1/models/gemini-1.5-pro:generateContent",
    headers=headers,  # Здесь headers определен ранее
    data=json.dumps(payload_summary)
)

# Обработка итогового обобщения
if response_summary.status_code == 200:
    summary = response_summary.json()['candidates'][0]['content']['parts'][0]['text']

    # Сохранение итогового обобщения
    with open(os.path.join(data_folder, 'summary_themes_gemini.txt'), 'w', encoding='utf-8') as f:
        f.write(summary)

    print("Анализ завершен. Промежуточные результаты сохранены в 'intermediate_themes_gemini.txt', итоговое обобщение сохранено в 'summary_themes_gemini.txt'.")
else:
    print(f"Ошибка при обобщении: {response_summary.status_code}, {response_summary.text}")

DataFrame сохранен в файл 'diary_dataframe.csv'.
Обработка файла 01.txt (1/9):
Обработка файла 02.txt (2/9):
Обработка файла 03.txt (3/9):
Обработка файла 04.txt (4/9):
Обработка файла 05.txt (5/9):
Обработка файла 06.txt (6/9):
Обработка файла 07.txt (7/9):
Обработка файла 08.txt (8/9):
Обработка файла 09.txt (9/9):
Анализ завершен. Промежуточные результаты сохранены в 'intermediate_themes_gemini.txt', итоговое обобщение сохранено в 'summary_themes_gemini.txt'.
